In [ ]:
import os
os.environ["TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL"] = "1"

In [ ]:
from sentence_transformers import CrossEncoder

# Qwen3-Reranker 不兼容 sentence-transformers 的 CrossEncoder API
# 使用 BAAI 的 bge-reranker-base（国产，支持中文，兼容 CrossEncoder）
model = CrossEncoder("BAAI/bge-reranker-v2-m3", device="cuda")

In [ ]:
query = "Python怎么读取文件？"

corpus = [
    "Python可以使用open()函数读取文件，配合read()或readlines()方法获取内容。",
    "Java通过BufferedReader类来读取文本文件的内容。",
    "Python是一种广泛使用的编程语言，由Guido van Rossum创建。",
    "使用pandas的read_csv()函数可以方便地读取CSV格式的数据文件。",
    "今天天气真不错，适合出去散步。",
]

In [ ]:
ranks = model.rank(query, corpus)

In [ ]:
ranks

In [ ]:
sentence_combinations = [[query, sentence] for sentence in corpus]

scores = model.predict(sentence_combinations)

In [ ]:
scores

vllm serve BAAI/bge-reranker-v2-m3 \
  --served-model-name bge-reranker-v2-m3 \
  --port 8001 \
  --dtype bfloat16 \
  --max-model-len 8192

http://117.50.197.225:8001/v1/score

{
    "model": "bge-reranker-v2-m3",
    "text_1": "什么是深度学习",
    "text_2": ["深度学习是机器学习的一个分支", "今天天气不错"]
}

{
    "id": "score-97bd6be0a4c58e08",
    "object": "list",
    "created": 1771664139,
    "model": "bge-reranker-v2-m3",
    "data": [
        {
            "index": 0,
            "object": "score",
            "score": 0.9988622665405273
        },
        {
            "index": 1,
            "object": "score",
            "score": 1.833783790061716e-05
        }
    ],
    "usage": {
        "prompt_tokens": 28,
        "total_tokens": 28,
        "completion_tokens": 0,
        "prompt_tokens_details": null
    }
}

vllm serve Qwen/Qwen3-Reranker-0.6B \
  --served-model-name qwen3-reranker \
  --port 8002 \
  --dtype bfloat16 \
  --max-model-len 8192

http://117.50.197.225:8002/v1/chat/completions

{
    "model": "qwen3-reranker",
    "messages": [
        {
            "role": "system",
            "content": "Judge whether the Document meets the requirements based on the Query and the Instruct provided. Note that the answer can only be \"yes\" or \"no\"."
        },
        {
            "role": "user",
            "content": "<Instruct>: Given a web search query, retrieve relevant passages that answer the query\n<Query>: 什么是深度学习\n<Document>: 深度学习是机器学习的一个分支，它使用多层神经网络来学习数据的表示"
        }
    ],
    "max_tokens": 1,
    "temperature": 0,
    "logprobs": true,
    "top_logprobs": 5,
    "chat_template_kwargs": {"enable_thinking": false}
}
